# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
import PyPDF2 as ppdf
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT

try:
    import nest_asyncio
    nest_asyncio.apply()
except:
    pass

if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','AllRatMaxPos.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

Plot the definition of before after ...

In [ ]:
def plot_session_def(ax, sessionSlices= tuple()):
    eps=0
    xpos=ax.get_xticks()
    
    for sli in sessionSlices:
        assert isinstance(sli, slice), 'Bad Boy!'        
        stop=sli.stop
        if stop is None:
            stop=-1
        start=sli.start
        if start>=0:
            start+=1
#             stop+=1
        
        ax.axvspan(xmin=start+eps, xmax=stop-eps, zorder=-1, alpha=.8, ec= None,
                  color='xkcd:ivory')
        
    if len(sessionSlices)==3:
        for sli,String in zip(sessionSlices,['Before','Acute','Stable']):
            stop=sli.stop
            if stop is None:
                stop=-1
            start=sli.start
            if start>=0:
                start+=1
            
            mid=(start+stop)/2
            
            ax.text(mid,ax.get_ylim()[1],s=String,
                   fontsize='xx-small', ha='center',va='top')

---

plotting group time course

In [ ]:
def normalize_performance(a,nPre):
    """implement a normalizing function to operate on 'a' and nPre first point are baseline
       a: np.array: animal x session
    """
    meanBase= np.nanmedian(a[:,:nPre], axis=1, keepdims=True)
    
    out=a-meanBase
    return out

def plot_normalized_time_course(root, ax, Profiles, Animals=None, TaskParamToPlot='% good trials', shift=0,
                   nPre=5, nPost=15, **kwarg):

    #getting the data
    animalList,SessionDict=event_detect(root, Profiles[0], Profiles[1])
    
    if Animals is None:
        Animals=animalList
    
    SessionDict={animal:SessionDict[animal] for animal in Animals}
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                      SessionDict,
                                                      parameter=param,
                                                      redo=False,
                                                      TaskParamToPlot=[TaskParamToPlot])
    
    assert nPre<=nSessionPre and nPost<=nSessionPost,"fewer sessions available than requested:"
    
    data=np.array(list(Results[TaskParamToPlot].values()))
    
    xData=np.append(np.arange(-nPre,0),np.arange(1,nPost+1))
    data=data[:,nSessionPre-nPre:nSessionPre+nPost]
    
    data=normalize_performance(data,nPre)
    
    reliableAnimals=np.sum(np.isnan(data),axis=1) < data.shape[1] / 2
    
    data=data[reliableAnimals,:]
    animalList=np.array(list(Results[TaskParamToPlot].keys()))[reliableAnimals]
    
    groupData=np.nanmedian(data,axis=0)
    
    groupErr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(xData + shift, groupData,
                abs(groupErr-groupData),
                fmt='-o', zorder=5, ms=2, elinewidth=1,alpha=.8, **kwarg)
    
    ax.axhline(0, ls=':',c='gray',lw=1)
    
    def _tik(x,pos):
        if x in [-nPre,-1,1,nPost]:
            return ('$%+g$' if x>0 else '$%g$')%x
        else:
            return ''
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(_tik))
    
    ax.set_xlim([-nPre-.3,nPost+.3])
    ax.spines['bottom'].set_bounds(-nPre,nPost)
    ax.set_xticks(xData)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
#     ax.set_ylim([0,100])
    ax.set_ylabel('Norm. '+TaskParamToPlot)
    ax.set_xlabel('Session # relative to lesion')

    return data,animalList


def plot_normalized_time_course_MoreSession(root, ax, Profiles, Animals=None, TaskParamToPlot='% good trials', shift=0,
                   nPre=5, nPost=15, **kwarg):

    #getting the data
    animalList,SessionDict=event_detect(root, Profiles[0], Profiles[1])
    
    if Animals is None:
        Animals=animalList
    
    SessionDict={animal:SessionDict[animal] for animal in Animals}
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                      SessionDict,
                                                      parameter=param,
                                                      redo=False,
                                                      TaskParamToPlot=[TaskParamToPlot])
    
    assert nPre<=nSessionPre and nPost<=nSessionPost,"fewer sessions available than requested:"
    
    data=np.array(list(Results[TaskParamToPlot].values()))
    
    xData=np.append(np.arange(-nPre,0),np.arange(1,nPost+1))
    data=data[:,nSessionPre-nPre:nSessionPre+nPost]
    
    data=normalize_performance(data,nPre)
    
    reliableAnimals=np.sum(np.isnan(data),axis=1) < data.shape[1] -2
    
    data=data[reliableAnimals,:]
    animalList=np.array(list(Results[TaskParamToPlot].keys()))[reliableAnimals]
    
    groupData=np.nanmedian(data,axis=0)
    
    groupErr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(xData + shift, groupData,
                abs(groupErr-groupData),
                fmt='-o', zorder=5, ms=2, elinewidth=1,alpha=.8, **kwarg)
    
    ax.axhline(0, ls=':',c='gray',lw=1)
    
    def _tik(x,pos):
        if x in [-nPre,-1,1,nPost]:
            return ('$%+g$' if x>0 else '$%g$')%x
        else:
            return ''
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(_tik))
    
    ax.set_xlim([-nPre-.3,nPost+.3])
    ax.spines['bottom'].set_bounds(-nPre,nPost)
    ax.set_xticks(xData)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
#     ax.set_ylim([0,100])
    ax.set_ylabel('Norm. '+TaskParamToPlot)
    ax.set_xlabel('Session # relative to lesion')

    return data,animalList



def add_legend_for_rat_groups(ax,N):
    g_marker = matplotlib.lines.Line2D([0,0], [1,1], color='gray',lw=1,
                                       markeredgecolor='gray', marker='o',
                                       markerfacecolor='gray', markersize=2, label=f'$\Delta$Speed$>0$\n($n={N[0]}$)')
    b_marker = matplotlib.lines.Line2D([0,0], [1,1], color='k',
                                       markeredgecolor='k', marker='o',lw=1,
                                       markerfacecolor='k', markersize=2, label=f'$\Delta$Speed$<0$\n($n={N[1]}$)')


    leg=ax.legend(handles=[b_marker,g_marker],loc=(0,.9),ncol=2,
                  facecolor=None,edgecolor=None, fontsize=5,frameon=False)
    return leg

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }
    profile2={'Type': ['Good'],
             'rewardType': ['Progressive'],
             'option': ['not used', 'AsymmetricLesion'],
             'initialSpeed': ['0', '10'],
             'Speed': ['10'],
             'Tag': ['Late-Lesion_DLS',
              'Late-Lesion_DMS-Sharp',
              'Late-Lesion_DMS',
              'Late-Lesion_DS',
              'Late-Lesion_DS-Sharp']}

    #number of sessions to plot
    animalList,_=event_detect(root,profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

#     a=plot_normalized_time_course(root, ax, Profiles,Animals=animalList,TaskParamToPlot=TaskParamToPlot,color='k',shift=-.1*.25)
    add_legend_for_rat_groups(ax,(5,1))

---

plot the speed correlation plot

In [ ]:
def _late_lesion_effect(root, Profiles, badAnimals, TaskParamToPlot:str,
                        preSlice=slice(-5,None), postSlice=slice(0,5)):
    
    _,sessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    [sessionDict.pop(key, None) for key in badAnimals]
    
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                 sessionDict,
                                                 parameter=param,
                                                 redo=False,
                                                 TaskParamToPlot=[TaskParamToPlot])

    data=np.array(list(Results[TaskParamToPlot].values()))
    slicedPreData  = np.ones((data.shape))*np.nan
    slicedPostData = np.ones((data.shape))*np.nan
    
    for row,Dnan in enumerate(data):
        Dpre= Dnan[:nSessionPre]#[~ np.isnan(Dnan[:nSessionPre])]
        Dpost=Dnan[nSessionPre:]#[~ np.isnan(Dnan[nSessionPre:])]
        preData =Dpre[preSlice]
        postData=Dpost[postSlice]

        slicedPreData[row,:len(preData)]=preData
        slicedPostData[row,:len(postData)]=postData
    
    yPre =np.nanmean(slicedPreData ,axis=1)
    yPost=np.nanmean(slicedPostData,axis=1)

    behav=[]
    animals=[]
    for i,animal in enumerate(Results[TaskParamToPlot].keys()):
        if np.isnan(yPre[i]) or np.isnan(yPost[i]):
            logging.error(f'{animal}: {TaskParamToPlot} not defined!')
            continue

        behav.append(yPost[i]-yPre[i])
        animals.append(animal)
    
    return behav, animals

def late_lesion_correlation_with_size(root, ax, Profiles, Animals, color, TaskParamToPlot:str,
                                      preSlice=slice(-5,None), postSlice=slice(0,5), Excluded='RatBAD'):
    
    _,sessionDict=event_detect(root, Profiles[0], Profiles[1])
    
    sessionDict={animal:sessionDict[animal] for animal in Animals}
    
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                 sessionDict,
                                                 parameter=param,
                                                 redo=False,
                                                 TaskParamToPlot=[TaskParamToPlot])

    data=np.array(list(Results[TaskParamToPlot].values()))
    slicedPreData  = np.ones((data.shape))*np.nan
    slicedPostData = np.ones((data.shape))*np.nan
    
    for row,Dnan in enumerate(data):
        Dpre= Dnan[:nSessionPre]#[~ np.isnan(Dnan[:nSessionPre])]
        Dpost=Dnan[nSessionPre:]#[~ np.isnan(Dnan[nSessionPre:])]
        preData =Dpre[preSlice]
        postData=Dpost[postSlice]

        slicedPreData[row,:len(preData)]=preData
        slicedPostData[row,:len(postData)]=postData
    
    yPre =np.nanmean(slicedPreData ,axis=1)
    yPost=np.nanmean(slicedPostData,axis=1)

    size=[]
    behav=[]
    animals=[]
    for i,animal in enumerate(Results[TaskParamToPlot].keys()):
        if np.isnan(yPre[i]) or np.isnan(yPost[i]):
            logging.error(f'{animal}: {TaskParamToPlot} not defined!')
            continue
        try:
            Hist=HistologyExcel('/NAS02',animal)
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        
        behav.append(yPost[i]-yPre[i])
        size.append(Hist.lesion_size())
        animals.append(animal)
        
    
    #plotting
    if ax is not None:
        for i,animal in enumerate(animals):
            if animal == Excluded: continue
            _,tag=lesion_type(root,animal)
            ax.scatter(size[i], behav[i], s=5, c=color[tag]);
#             ax.annotate(animal[-3:], (size[i],behav[i]), fontsize='xx-small')

        ax.set_xticks(np.arange(0,1.01,.1))
        ax.set_xticklabels(['0','','','','','0.5','','','','','1'])
        ax.spines['bottom'].set_bounds(0,1)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlim([-.02,1.02])
        ax.set_ylabel(f'$\Delta$ {TaskParamToPlot}')
        ax.set_xlabel('Lesion size')

    
    return behav, size, animals

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
             }

    Profiles=(profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    preSlice=slice(-5,None)
    postSlice=slice(3,8)
    
    color={'DS':'r','DMS':'g', 'DLS':'b', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    
#     a=late_lesion_correlation_with_size(root, ax=ax, Profiles=Profiles,color=color, TaskParamToPlot=TaskParamToPlot,
#                                       preSlice=preSlice, postSlice=postSlice)

---

plotting group double errorbars for speed 

In [ ]:
def plot_event_1on1(root, ax, profile, color, TaskParamToPlot='% good trials',
                   x_pos=1, nSlice=slice(-5,None), seed=1, animal_plot=True):
    
    animalList=batch_get_animal_list(root,profile)
    
    data=[]
    for animal in animalList:
        out=animal_learning_stats(root, animal, TaskParamToPlot, profile, nSlice)
        if len(out)==0:
            continue
        data.append(out)

    
    data=np.array(data)
    data=np.nanmedian(data,axis=1)
    data=data[np.logical_not(np.isnan(data))]
    
    #plotting individual animals
    if animal_plot:
        ax.boxplot(x=data,whis=[5,95],
              positions=[x_pos], widths=.3,
              showcaps=False, showfliers=False,
              medianprops={'color':color}, boxprops={'color':color}, whiskerprops={'color':color},
              zorder=1)
        
        
#     ax.set_xlim([x_pos[0]-diff/8,x_pos[-1]+diff/8])
#     ax.set_xticks(x_pos)
#     ax.set_xticklabels(['Before','Stable'])
#     ax.xaxis.set_tick_params(rotation=70)
#     ax.spines['bottom'].set_bounds(x_pos[0],x_pos[-1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
#     ax.set_ylabel(TaskParamToPlot)
#     ax.set_xlabel('Sessions relative to lesion')

    return data

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
                     'rewardType':'Progressive',
                     'initialSpeed':['10'],
                     'Speed':'10',
                     'Tag':['Control', 'Control-Sharp']
                      }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DMS-Sharp','Late-Lesion_DLS','Late-Lesion_DS-Sharp']
             }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot="Front Starting Trials"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    ax.set_xticks([])
    data=plot_event_1on1(root, ax, profile1, color='k', TaskParamToPlot="Maximum Position",
                   x_pos=1, nSlice=slice(25,30), seed=1, animal_plot=True)
    
    

---

plotting group lesion size

In [ ]:
def plot_group_lesion_bar(animalList, ax=None, x=0, color='k'):
    _W=.8   #bar width
    lesionsize=[]
    for animal in animalList:
        try:
            tmp=HistologyExcel('/NAS02',animal).lesion_size()
        except:
            tmp=np.nan
        finally:
            lesionsize.append(tmp)
    
    if ax is not None:
        ax.bar(x,np.nanmedian(lesionsize), ec=None, fc=color, zorder=1)
        
        _coeff=2
        x_vals=np.random.uniform(low=x-_W/_coeff, high=x+_W/_coeff, size=len(animalList))
        ax.scatter(x_vals, lesionsize, s=3, c='xkcd:red', marker='o', edgecolors='none', alpha=.8, zorder=2)
        np.random.seed(seed=3)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.set_xlim([x-_W,x+_W])
        ax.set_xticks([x])
        
    return np.array(lesionsize)

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    plot_group_lesion_bar(['Rat304','Rat302','Rat079'],ax=ax)

---

Stef's functions

In [ ]:
def add_legend_trajectory(ax,cmap,ratio):
    
    markers = []
    
    for i in range(ratio.shape[0]):
        marker = matplotlib.lines.Line2D([0], [0],c=cmap(2*i),label=r'$\alpha=$' + str(ratio[i]))
        markers.append(marker)
    
    
    leg=ax.legend(handles=markers,loc=(0,1), ncol=6, fontsize='xx-small', framealpha=0,mode='expand')
    
    return leg

def penalty(x,k,mu,amp):
    '''This function generates a Heaviside step function of steepness k, centered around mu and of amplitude amp'''
    return(amp/(1+np.exp(2*k*(x-mu))))


def compute_cost_trajectories(trajectory, speed, force, a, b, cost_type, 
                              vtapis, Ltread, xb, 
                              kxpenalty, ampxpenalty):
    
    n_trajectoires = trajectory.shape[1]
    inst_cost = np.zeros(trajectory.shape)
    
    if cost_type == 'speed_quadratic':
        for i_t in range(n_trajectoires):
            inst_cost[:,i_t] = a*(speed[i_t]-vtapis)**2 + b*(trajectory[i_t] - Ltread)**2
    
    elif cost_type == 'force_quadratic':   
        for i_t in range(n_trajectoires):
            inst_cost[:,i_t] = a*(force[i_t]-vtapis)**2 + b*(trajectory[i_t] - Ltread)**2
        
    elif cost_type == 'speed_heaviside':
        for i_t in range(n_trajectoires):
            inst_cost[:,i_t] = a*(speed[:,i_t]-vtapis)**2 
            + b*penalty(x=trajectory[:,i_t], k=kxpenalty, mu=xb, amp=ampxpenalty)       
    
    elif cost_type == 'force_heaviside':   
        for i_t in range(n_trajectoires):
            inst_cost[:,i_t] = a*(force[:,i_t]-vtapis)**2 
            + b*penalty(x=trajectory[i_t], k=kxpenalty, mu=xb, amp=ampxpenalty)
    return inst_cost

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='k'
    DLSColor='xkcd:orange'
    DMSColor='purple'
    DSColor='xkcd:green'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    colorSig='goldenrod'
    
    
    #===============================================
    
    # GRID 5 PARAMS
    
    TaskParamToPlot5="standard deviation of entrance time"
    nPre5=5
    nPost5=10
    _s=.15
    ShiftDLS5 =-_s
    ShiftDMS5 =0*_s
    ShiftDS5  =_s
    
    
    
    profilePreLesion5={'Type':'Good',
                     'rewardType':'Progressive',
                     'initialSpeed':['10'],
                     'Speed':'10',
                     'Tag':['Control', 'Control-Sharp']
                      }
    
    profileDLS5={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DLS']
                 }
    profileDMS5={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DMS','Late-Lesion_DMS-Sharp'],
                 }  
    profileDS5={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DS','Late-Lesion_DS-Sharp']
                 }  

    ProfilesDLS5=(profilePreLesion5,profileDLS5,)
    ProfilesDMS5=(profilePreLesion5,profileDMS5,)
    ProfilesDS5 =(profilePreLesion5,profileDS5,)
    
    
    #================================================
    
    # GRID 6 PARAMS

    profileLesions6={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': list(set((*profileDLS5['Tag'], *profileDMS5['Tag'], *profileDS5['Tag'])))
                     }
    profileLesions6['Tag'].extend(['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS',
                                   'Early-Lesion_DS-NoHab','Early-Lesion_DMS-NoHab'])

    Profiles6=(profilePreLesion5,profileLesions6)
    preSlice6Ctrl=slice(25,30)
    preSlice6=slice(-5,None)
    TaskParamToPlot6="Maximum Position Relxed"
    

    #================================================
    
    # GRID 1 PARAMS

    profileLesions1={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': list(set((*profileDLS5['Tag'], *profileDMS5['Tag'], *profileDS5['Tag'])))
                     }
    profilePreLesion1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }
    

    Profiles1=(profilePreLesion1,profileLesions1)
    
    preSlice1=slice(-5,None)
    postSlice1=slice(0,3)
    finSlice1=slice(8,13)
    
    minSpdReduction1=0
    
    TaskParamToPlot1="Forward Running Speed"
    

Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params({'axes.labelsize':'x-small'})
    figsize=(5,5)
    fig=plt.figure(figsize=figsize,dpi=600)
    
   
    
##=====================================================================
##===================STEF's CODE BELOW=================================
##=====================================================================
    baseFolder=os.path.dirname(currentNbPath)
    # Panel A and B
    simulations_results_folder = baseFolder+'/PickleResults/Simulations'
    diffuse_heaviside_effort_file = simulations_results_folder + '/DiffuseHeavisideSpatialCostEffort.pickle'
    file_to_open = diffuse_heaviside_effort_file
    with open(file_to_open, 'rb') as handle:
        results_simulations = pickle.load(handle)  

    xk_vect_speed_heav = results_simulations['xk_vect_speed_heav']
    tk = results_simulations['tk_vect_speed_heav'][:,0]
    xdotk_vect_speed_heav = results_simulations['xdotk_vect_speed_heav']
    acck_vect_speed_heav = results_simulations['acck_vect_speed_heav']
    # Panel C and D
    data_folder = baseFolder+'/Data_for_fit/Before/'
    file_to_open = data_folder + 'clean_max_pos_traj.pickle'
    with open(file_to_open, 'rb') as handle:
        trajectories_before = pickle.load(handle)    

    data_folder = baseFolder+'/Data_for_fit/Final/'
    file_to_open = data_folder + 'clean_max_pos_traj.pickle'
    with open(file_to_open, 'rb') as handle:
        trajectories_final = pickle.load(handle)
    file_to_open = data_folder + '_lesion_size_.p'   
    with open(file_to_open, 'rb') as handle:
        lesion_size_final = pickle.load(handle)    

    fit_results_folder = baseFolder+'/PickleResults/Fit/'
    # Before Lesion: effort parameter fitted trajectory
    with open(fit_results_folder + 'a_fit_bounded_beforeXX.pickle', 'rb') as handle:
        results_a_bounded_before = pickle.load(handle) 
    with open(fit_results_folder + 'xk_fit_bounded_beforeXX.pickle', 'rb') as handle:
        results_xk_bounded_before = pickle.load(handle)     
    # After Lesion: effort parameter fitted trajectory
    with open(fit_results_folder + 'a_fit_bounded_finalXX.pickle', 'rb') as handle:
        results_a_bounded_final = pickle.load(handle) 
    with open(fit_results_folder + 'xk_fit_bounded_finalXX.pickle', 'rb') as handle:
        results_xk_bounded_final = pickle.load(handle)  
    with open(fit_results_folder + 'delta_a_boundedXX.pickle', 'rb') as handle:
        delta_a_median_bounded_dict = pickle.load(handle)


    simulations_results_folder = baseFolder+'/PickleResults/Simulations'
    steep_heaviside_effort_names_list = [simulations_results_folder + '/SteepHeavisideSpatialCostEffort.pickle']
    diffuse_heaviside_effort_names_list = [simulations_results_folder + '/DiffuseHeavisideSpatialCostEffort.pickle']
    steep_heaviside_mass_names_list = [simulations_results_folder + '/SteepHeavisideSpatialCostMass.pickle']
    diffuse_heaviside_mass_names_list = [simulations_results_folder + '/DiffuseHeavisideSpatialCostMass.pickle']

    # Load desired data and define relevant variables
    file_to_open = steep_heaviside_effort_names_list[0]
    with open(file_to_open, 'rb') as handle:
        results = pickle.load(handle)  
    xk_vect_speed_heav_steep_effort = results['xk_vect_speed_heav']
    xk_vect_force_heav_steep_effort = results['xk_vect_force_heav']

    file_to_open = diffuse_heaviside_effort_names_list[0]
    with open(file_to_open, 'rb') as handle:
        results = pickle.load(handle)  
    xk_vect_speed_heav_diffuse_effort = results['xk_vect_speed_heav']
    xk_vect_force_heav_diffuse_effort = results['xk_vect_force_heav']


    file_to_open = steep_heaviside_mass_names_list[0]
    with open(file_to_open, 'rb') as handle:
        results = pickle.load(handle)  
    xk_vect_speed_heav_steep_mass = results['xk_vect_speed_heav']
    xk_vect_force_heav_steep_mass = results['xk_vect_force_heav']

    file_to_open = diffuse_heaviside_mass_names_list[0]
    with open(file_to_open, 'rb') as handle:
        results = pickle.load(handle)  
    xk_vect_speed_heav_diffuse_mass = results['xk_vect_speed_heav']
    xk_vect_force_heav_diffuse_mass = results['xk_vect_force_heav']

    tk = results['tk_vect_speed_heav'][:,0] # this is independent on the model


    aux_a = [.1,1,2,5,10,100] 
    NTRAJECTORIES = len(aux_a) # I get rid of the alpha=2000
    cmap = matplotlib.cm.get_cmap('jet', 2*NTRAJECTORIES)

    ##############################################################

    nrows = 1
    ncols = 4

    linewidth = 1.5
    axes1 = []
    ##################################################################################################
    gs = fig.add_gridspec(nrows=nrows, ncols=ncols, left=0, bottom=0.84, right=.88, top=1.,wspace=0.15,hspace=0.3)
    ##################################################################################################
    # FIRST PANEL
    ax=fig.add_subplot(gs[0])
    axes1.append(ax)

    for it in range(NTRAJECTORIES):
        ax.plot(tk,xk_vect_speed_heav_steep_effort[:,it],linewidth=linewidth,alpha=1.0,c=cmap(2*it))

    ax.text(0.3,.78,'Spatial cost $=$ Localized \nEffort' + r'$\approx$' + 'Kinetic energy',fontsize=4)
    #################################################
    # SECOND PANEL    
    ax=fig.add_subplot(gs[1])
    axes1.append(ax)

    for it in range(NTRAJECTORIES):
        ax.plot(tk,xk_vect_speed_heav_diffuse_effort[:,it],linewidth=linewidth,alpha=1.0,c=cmap(2*it))

    ax.text(0.3,.78,'Spatial cost $=$ Diffuse \nEffort' + r'$\approx$' + 'Kinetic energy',fontsize=4)
    #################################################
    # THIRD PANEL
    ax=fig.add_subplot(gs[2])
    axes1.append(ax)

    for it in range(NTRAJECTORIES):
        ax.plot(tk,xk_vect_force_heav_steep_effort[:,it],linewidth=linewidth,alpha=1.0,c=cmap(2*it))

    ax.text(0.3,.78,'Spatial cost $=$ Localized \nEffort' + r'$\approx$' + 'Force$^2$',fontsize=4)
    ax.text(.44,1.02,'Fixed weight',fontsize=7,ha='center', transform=fig.transFigure)
    #################################################
    # FOURTH PANEL
    ax=fig.add_subplot(gs[3])
    axes1.append(ax)

    for it in range(NTRAJECTORIES):
        ax.plot(tk,xk_vect_force_heav_diffuse_effort[:,it],linewidth=linewidth,alpha=1.0,c=cmap(2*it))

    ax.text(0.3,.78,'Spatial cost $=$ Diffuse \nEffort' + r'$\approx$' + 'Force$^2$',fontsize=4)
    #############################################
    # COLORMAP
    gsc = fig.add_gridspec(nrows=1,ncols=1,left=.92,bottom=0.84,top=1,right=1)
    axc = fig.add_subplot(gsc[0])
    axc.xaxis.set_visible(False)
    axc.spines['left'].set_visible(False)
    axc.spines['bottom'].set_visible(False)
    axc.spines['top'].set_visible(False)
    axc.spines['right'].set_visible(False)
    axc.tick_params(color=(0, 0, 0, 0),labelcolor=(0, 0, 0, 0),zorder=-10)

    c = np.arange(1, 50*NTRAJECTORIES + 1)
    cmap_ = matplotlib.cm.get_cmap('jet', 50*NTRAJECTORIES)
    dummie_cax = axc.scatter(c, c, c=c, cmap=cmap_)
    # Clear axis
    axc.cla()

    cbaxes = inset_axes(axc, width="20%", height="60%",loc=3) 
    shift = 50
    cb=fig.colorbar(dummie_cax,cax=cbaxes,ticks=[1+shift,50*NTRAJECTORIES],aspect=10)
    cb.outline.set_edgecolor(None)
    cb.set_label('Effort\nSensitivity', labelpad=-13,y=1.5, rotation=0,fontsize='xx-small')

    cb.ax.set_yticklabels(['Low', 'High'],rotation=45,fontsize=4)
    cb.ax.yaxis.set_tick_params(size=0)
    #############################################

    ##################################################################################################
    gs = fig.add_gridspec(nrows=nrows, ncols=ncols, left=0, bottom=.58, right=.88, top=.74,wspace=0.15,hspace=0.3)
    ##################################################################################################
    # FIRST PANEL
    ax=fig.add_subplot(gs[0])
    axes1.append(ax)

    for it in range(NTRAJECTORIES):
        ax.plot(tk,xk_vect_speed_heav_steep_mass[:,it],linewidth=linewidth,alpha=1.0,c=cmap(2*it))

    ax.set_xlabel('Time (s)',labelpad=-5)
    #################################################
    # SECOND PANEL    
    ax=fig.add_subplot(gs[1])
    axes1.append(ax)

    for it in range(NTRAJECTORIES):
        ax.plot(tk,xk_vect_speed_heav_diffuse_mass[:,it],linewidth=linewidth,alpha=1.0,c=cmap(2*it))
    #################################################
    # THIRD PANEL
    ax=fig.add_subplot(gs[2])
    axes1.append(ax)

    for it in range(NTRAJECTORIES):
        ax.plot(tk,xk_vect_force_heav_steep_mass[:,it],linewidth=linewidth,alpha=1.0,c=cmap(2*it))

    ax.text(.44,.76,'Fixed effort sensitivity',fontsize=7,ha='center', transform=fig.transFigure)
    #################################################
    # FOURTH PANEL
    ax=fig.add_subplot(gs[3])
    axes1.append(ax)

    for it in range(NTRAJECTORIES):
        ax.plot(tk,xk_vect_force_heav_diffuse_mass[:,it],linewidth=linewidth,alpha=1.0,c=cmap(2*it))
    #############################################
    # COLORMAP
    gsc = fig.add_gridspec(nrows=1,ncols=1,left=.92,bottom=.58,top=.74,right=1)
    axc = fig.add_subplot(gsc[0])
    axc.xaxis.set_visible(False)
    axc.spines['left'].set_visible(False)
    axc.spines['bottom'].set_visible(False)
    axc.spines['top'].set_visible(False)
    axc.spines['right'].set_visible(False)
    axc.tick_params(color=(0, 0, 0, 0),labelcolor=(0, 0, 0, 0),zorder=-10)

    c = np.arange(1, 50*NTRAJECTORIES + 1)
    cmap_ = matplotlib.cm.get_cmap('jet', 50*NTRAJECTORIES)
    dummie_cax = axc.scatter(c, c, c=c, cmap=cmap_)
    # Clear axis
    axc.cla()

    cbaxes = inset_axes(axc, width="20%", height="60%",loc=3) 
    shift = 50
    cb=fig.colorbar(dummie_cax,cax=cbaxes,ticks=[1+shift,50*NTRAJECTORIES],aspect=10)
    cb.outline.set_edgecolor(None)
    cb.set_label('Rat\'s\nWeight', labelpad=-13,y=1.5, rotation=0,fontsize='xx-small')
    cb.ax.set_yticklabels(['Low', 'High'],rotation=45,fontsize=4)
    cb.ax.yaxis.set_tick_params(size=0)
    #################################################

    ###############################################################################
    for ind_ax,ax in enumerate(axes1):

        ax.plot(tk,xk_vect_speed_heav[0,0]+[0.1*t for t in tk],'--',color='Black',alpha=.8,zorder=-1,linewidth=1)
        ax.plot(tk,[0.1 for t in tk],':',color='Gray',zorder=-1,linewidth=1)

        if ind_ax==4:
            ax.set_ylabel('Position (cm)')    
            ax.set_yticklabels(['0'] + ['' for i in range(8)] + ['90'])
            ax.set_xlabel('Time (s)',labelpad=-5)
        else:
            ax.set_yticks([0.1*i for i in range(10)])
            ax.set_yticklabels(['' for i in range(10)])

        ax.set_yticks([0.1*i for i in range(10)])
        ax.set_xticks([i for i in range(8)])

        if ind_ax==4:
            ax.set_xticklabels(['0'] + ['' for i in range(6)] + ['7'])
        else:
            ax.set_xticklabels(['' for i in range(8)])

        ax.set_xlim(0,7.5)
        ax.set_ylim(0,0.9)
        ax.spines['bottom'].set_bounds(0,7)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)    


##=====================================================================
##===================STEF's CODE ABOVE=================================
##=====================================================================

    
    
    ##########################################
    # 3: Normalized speed time course
    gs3= fig.add_gridspec(nrows=1, ncols=1, left=.24, bottom=.28, right=0.59, top=.48)
    ax3= fig.add_subplot(gs3[0])
    
    
    behav3, AllAnimals3=_late_lesion_effect(root, Profiles=Profiles1,badAnimals=[],TaskParamToPlot=TaskParamToPlot1,
                                            preSlice=preSlice1, postSlice=finSlice1)

    goodAnimals3=[animal for i,animal in enumerate(AllAnimals3) if behav3[i]<minSpdReduction1]
    badAnimals3=list(set(AllAnimals3) - set(goodAnimals3))
    
    data3G,goodAnimals3=plot_normalized_time_course(root, ax3, Profiles1,Animals=goodAnimals3,TaskParamToPlot=TaskParamToPlot1,color='k',shift=-.1*.5)
    data3B,_=plot_normalized_time_course(root, ax3, Profiles1,Animals=badAnimals3,TaskParamToPlot=TaskParamToPlot1,color='gray',shift=.1*.5)
    

    add_legend_for_rat_groups(ax3,(data3B.shape[0],data3G.shape[0]))
    
    ax3.set_yticks(range(-30,11,10))
    ax3.set_ylabel('Norm. speed (cm/s)',labelpad=0)
    ax3.spines['left'].set_bounds(-30,10)
    ax3.set_ylim([-35,12])
    
    
    
    ##########################################
    # 6: % Lesion size cmparison
    gs4= fig.add_gridspec(nrows=1, ncols=1, left=.66, bottom=0.28, right=0.76, top=.48)
    ax4= fig.add_subplot(gs4[0])
    
    
    plot_group_lesion_bar(goodAnimals3, ax4, x=1, color='k')
    plot_group_lesion_bar(badAnimals3 , ax4, x=2, color='gray')
    

    
    
    ax4.set_xlim([.5,2.5])
    ax4.set_xticks([1,2])
    ax4.set_xticklabels(['$\Delta$Speed$<0$','$\Delta$Speed$>0$'], fontsize='xx-small', rotation=15)
    ax4.set_ylim([0,1])
    ax4.set_yticks(np.arange(0,1.01,.1))
    ax4.set_yticklabels([0,'','','','','','','','','',1])
    ax4.set_ylabel('Lesion size',labelpad=-2)
    
    
    ##########################################
    # 6: ALL rat comparison
    gs6= fig.add_gridspec(nrows=1, ncols=1, left=.38, bottom=0, right=.62, top=.16)
    ax6= fig.add_subplot(gs6[0])
    
    data6 =plot_event_1on1(root, ax6, Profiles6[0], color=CtrlColor, TaskParamToPlot=TaskParamToPlot6,
                   x_pos=0, nSlice=preSlice6Ctrl)
    data6_=plot_event_1on1(root, ax6, Profiles6[1], color=DMSColor, TaskParamToPlot=TaskParamToPlot6,
                   x_pos=1, nSlice=preSlice6)

    
    

    ax6.set_xlim([-.4,1.4])
    ax6.set_ylim([50,85])
    ax6.spines['left'].set_bounds(50,85)
    ax6.spines['bottom'].set_bounds(0,1)
    ax6.set_yticks(np.arange(50,86,5))
    ax6.set_xticks([0,1])
    ax6.set_xticklabels([f'Control\n$n={len(data6)}$',f'Lesion\n$n={len(data6_)}$'])
    ax6.set_ylabel('Max. Pos. (cm)')

    
        
#   #STATS
    test6=permtest(group1=data6, group2=data6_, nIterations=10000)
    s=f'$p={test6.pVal[0]}$' if test6.significantDiff[0] else 'n.s.'
    s='p<0.0001' if test6.pVal[0]==0 else s
    test6.plotSigPair(ax6,y=84.5,x=(0,1), s=s, color=colorSig,lw=.8)    
    
    
    
    

    
#     fig.align_ylabels([ax2,ax4,ax6])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(axes1[0],axes1[4],ax3,ax4,ax6)
    OFFX=np.array([.07]*len(AXES))
    OFFY=np.array([.03]*len(AXES))

#     OFFX[[-3,-2,-1]]=0.06
#     OFFX[[3,2,1]]=0.04
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','AllRatMaxPos.pdf'),
                format='pdf', bbox_inches='tight')
    
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()